This is an [jupyter](http://jupyter.org) notebook.
Lectures about Python, useful both for beginners and experts, can be found at http://scipy-lectures.github.io.

Open the notebook by (1) copying this file into a directory, (2) in that directory typing 
jupyter-notebook
and (3) selecting the notebook.

***
Written By: **Riddhish Bhalodia**
***

In this exercise, we will learn about different neural network concepts. There are few prerequisites of probability and machine learning.

## The Perceptron Algorithm

It is one of the example of a linear discriminant model and used for two-class clustering / separation. In this model the input bector **x** is transformed using a fixed non-linear transformation. So starting from generalized model of linear regression we have

$$ y(\textbf{x}) = \textbf{w}^T\phi(\textbf{x})$$

Now in perceptron all we do is pass this linear regression model through a non-linear activation function as follows

$$y(\textbf{x}) = f(\textbf{w}^T\phi(\textbf{x})) \quad \quad \quad (1)$$

Here, $f(.)$ is given by 
$$
f(a) = \left\{
  \begin{array}{ll}
  -1 & \quad a < 0 \\
   1 & \quad a \geq 0
   \end{array}
   \right.
$$

Now, as we have two classes $\mathcal{C}_1$ and $\mathcal{C}_2$ so we define a target variable t which takes the values +1 and -1 for $\mathcal{C}_1$ and $\mathcal{C}_2$ respectively. Now we need to determine the parameters **w**, for that we need to define an error function which we have to minimize.

A natural choice for the error function is total number of misclassified patterns, however this causes some problems in the learning algorithm. Hence we propose an alternate error function called the *perceptron criterion* given by 

$$ E_p(\textbf{w}) = - \sum \limits _{n \in \mathcal{M}} \textbf{w}^T \phi_n t_n$$

Here, $\mathcal{M}$ denotes the set of all the misclassified patterns, the reasoning behind this functional can be found Christopher M Bishop's book [here](http://www.springer.com/gp/book/9780387310732) :D